# <font color= green> Pandas Analysis </font>

In [2]:
import pandas as pd
import numpy as np

In [3]:
filepath = './HeroesOfPymoli_Resources_purchase_data.csv'

In [4]:
df = pd.read_csv(filepath,encoding="ISO-8859-1")

In [5]:
#preview of DataFrame
df.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


In [6]:
#identify complete rows
df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [7]:
#review the column types
df.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [8]:
#change item id from Int64 to float64
df['Item ID'] = pd.to_numeric(df['Item ID'], downcast= 'float')
df['Purchase ID'] =pd.to_numeric(df['Purchase ID'], downcast= 'float')

df.dtypes

Purchase ID    float32
SN              object
Age              int64
Gender          object
Item ID        float32
Item Name       object
Price          float64
dtype: object

# <font color = green> Pandas Analysis: Heroes of Pymoli </font>

### Total Player Count

In [9]:
#Calculate Total Number of Players
playercount =df['SN'].nunique()
playercount

576

### Purchasing Analysis (Total)

In [10]:
#Calculate Number of Unique Items, Avg purch Price, Total Num of Puchases, & Total Revenue
pd.options.display.float_format = '{:,.2f}'.format
numofuniqueitems_df = round(df['Item Name'].nunique(),1)
#I know there is 2 item ids for 1 item , which I show later in the analysis. thus I used item name

avgpurchprice_df =round(df['Price'].mean(),1)


ttlnumofpurchases_df = round(df['Purchase ID'].count(),1)


totalrev_df = round(df['Price'].sum(),1)


#merge our data frames together
Purchasing_Analysis_df= pd.DataFrame({"Number of Unique Items": [numofuniqueitems_df], "Average Purchase Price":[avgpurchprice_df],"Total Number of Purchases": [ttlnumofpurchases_df],"Total Revenue":[totalrev_df]})
Purchasing_Analysis_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,3.10,780,"2,379.80"


### Gender Demographics

In [11]:
#Gender demographics
players = df.groupby('Gender')['SN'].nunique()

In [12]:
#to get the values for Gender Summary Table

#Calculating the % Count per Gender Bracket
maleplayers_perc = ((players.iloc[1]/playercount)*100)
femaleplayers_perc = (players.iloc[0]/playercount)*100
otherplayers_perc = (players.iloc[2]/playercount)*100

#Merge Data into a DF
genderdemo_df = pd.DataFrame([
    {"Description":"Male Players","Count": players.iloc[1],"Percentage": maleplayers_perc},
    {"Description":"Female Players","Count": players.iloc[0],"Percentage": femaleplayers_perc},
    {"Description":"Other Players","Count": players.iloc[2],"Percentage": otherplayers_perc}
])

format_mapping={'Percentage': '{:.2f}%'}
genderdemo_df.set_index('Description').reset_index()


,Description,Count,Percentage
0,Male Players,484,84.03
1,Female Players,81,14.06
2,Other Players,11,1.91


### Purchase Analysis (Gender)

In [13]:
Purchase_Count= df.groupby('Gender')['Price'].count()
Purchase_Count_set =Purchase_Count.reset_index(name='Total # Purchases')

In [14]:
#Purchase Analysis (Gender): Avg Purchase Price
avgpurprice = df.groupby('Gender')['Price'].mean()
avgpurprice_set = avgpurprice.reset_index(name='Avg Purchase Price')

In [15]:
#Purchase Analysis (Gender): Total Purchase Value
ttlpurvalue = df.groupby('Gender')['Price'].sum()
ttlpurvalue_set = ttlpurvalue.reset_index(name= 'Total Purchase Value')

In [16]:
#Purchase Analysis (Gender): Avg Purchase Total per Gender
avpurttl = df.groupby('Gender')['Price'].sum()/df.groupby('Gender')['SN'].nunique()
avpurttl_set = avpurttl.reset_index(name="Avg. Purchase Total")

In [17]:
#extra analysis: which gender purchased at a higher magnitude?
playercountbygender= round(Purchase_Count/players,1)
playercountbygender_df =playercountbygender.reset_index(name = "Purchases per Player")  

In [18]:
#Purchase Analysis (Gender): Final DataFrame 
purchase_anaysis_df = pd.merge(Purchase_Count_set,avgpurprice_set, how = 'outer', on="Gender")
purchase_anaysis_df_2 = pd.merge(purchase_anaysis_df,ttlpurvalue_set, how = 'outer', on="Gender")
purchase_anaysis_df_3 = pd.merge(purchase_anaysis_df_2,avpurttl_set, how = 'outer', on="Gender")
purchase_anaysis_df_4 = pd.merge(purchase_anaysis_df_3, playercountbygender_df, how = 'outer', on="Gender")
purchase_anaysis_df_4.sort_values(by = "Total # Purchases", ascending = False)

,Gender,Total # Purchases,Avg Purchase Price,Total Purchase Value,Avg. Purchase Total,Purchases per Player
1,Male,652,3.02,"1,967.64",4.07,1.30
0,Female,113,3.20,361.94,4.47,1.40
2,Other / Non-Disclosed,15,3.35,50.19,4.56,1.40


### Age Demographics

In [19]:
#create a new column that is filled with a description object based on the value in the age column
#create bins for the data to be held 
Age_Bins= [0,10,14,19,24,29,34,39,100]
#create the names for the for bins
brackets_descriptions= ['>10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

In [20]:
#to create new DF with Age Bracket column 
df["Age Bracket"] = pd.cut(df["Age"],Age_Bins, labels= brackets_descriptions)

In [21]:
#calculate Purchase Count
age_purch_count = df.groupby('Age Bracket')["Price"].count().reset_index(name = "Purchase Count")

In [22]:
#Calculate Average Purchase Price
age_avgpurch_price = df.groupby('Age Bracket')["Price"].mean().reset_index(name = "Average Purchase Price")

In [23]:
# Calculate Total Purchase Value
age_totalpurch_value = df.groupby('Age Bracket')["Price"].sum().reset_index(name = "Total Purchase Value")

In [24]:
#Calculate Average Purchase Total per Person by Age Group
age_avgtotalper_age = age_totalpurch_value.iloc[:,1]/age_purch_count.iloc[:,1]

In [25]:
#to make a df of Average Purchase Total per Person by Age Group
age_avgtotalper_age_df= pd.DataFrame({"Age Bracket":brackets_descriptions, "Average Purchase Total per Age Group": age_avgtotalper_age})

In [26]:
#Age Demographics: Final DataFrame 
age_Purchcount_avgpurchprice_df = pd.merge(age_purch_count,age_avgpurch_price, how = 'outer', on="Age Bracket")
age_totalpurch_df = pd.merge(age_Purchcount_avgpurchprice_df,age_totalpurch_value, how = 'outer', on="Age Bracket")
age_demographics_df = pd.merge(age_totalpurch_df,age_avgtotalper_age_df, how = 'outer', on="Age Bracket")
age_demographics_df

,Age Bracket,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Age Group
0,>10,32,3.40,108.96,3.40
1,10-14,19,2.68,50.95,2.68
2,15-19,136,3.04,412.89,3.04
3,20-24,365,3.05,"1,114.06",3.05
4,25-29,101,2.90,293.00,2.90
5,30-34,73,2.93,214.00,2.93
6,35-39,41,3.60,147.67,3.60
7,40+,13,2.94,38.24,2.94


In [27]:
#source[creating a new column & assign value]: https://stackoverflow.com/questions/49161120/pandas-python-set-value-of-one-column-based-on-value-in-another-column
#source[checkifavalueisbetween2values]:https://stackoverflow.com/questions/40156469/how-to-check-if-any-value-of-a-column-is-in-a-range-in-between-two-values-in-p

### Top 5 Spenders

In [38]:
#Top 5 spenders
#Top 5 Spenders: Identify Top Spenders:
spenders_df = df.groupby('SN')['Price'].sum().sort_values(ascending = False)
TotalPurchValue_df= spenders_df.iloc[0:4].reset_index(name='Total Purchase Value')

In [39]:
listoftopspenders = list(TotalPurchValue_df.iloc[:,0])

In [40]:
#Top 5 spenders: updated df
spenders_analysis_df= df.loc[df['SN'].isin(listoftopspenders)]

In [41]:
#Top 5 spenders: Average Purchase Price
Purchase_Count_df = spenders_analysis_df.groupby('SN')['Price'].count().sort_values(ascending = False).reset_index(name = "Purchase Count")

In [42]:
#Insert a new column in the orginal Purchase Price df to include the Avg. Purchase Price 
#Avg Purchase price is caculated by Total Purchase Value / Purchase Count

#to create a list of the variables for the dataframe
Avg_Purch_list= list(TotalPurchValue_df.iloc[0:4,1]/Purchase_Count_df.iloc[0:4,1])
SN_list= list(TotalPurchValue_df.iloc[0:4,0])

Average_Purch_Price_df=pd.DataFrame({"Avg. Purchase Price":Avg_Purch_list ,"SN":SN_list}).set_index('SN').reset_index()

In [43]:
#to merge dataframes
valueandcount_merg_df = pd.merge(TotalPurchValue_df,Purchase_Count_df, how = 'outer', on="SN")
top5spenders_df = pd.merge(valueandcount_merg_df,Average_Purch_Price_df, how = 'outer', on="SN")
top5spenders_df

,SN,Total Purchase Value,Purchase Count,Avg. Purchase Price
0,Lisosia93,18.96,5,3.79
1,Idastidru52,15.45,4,3.86
2,Chamjask73,13.83,3,3.46
3,Iral74,13.62,4,4.54


### Most Popular Items

In [33]:
#Table 1 of Item Name and Purchase Count 
listofpopulargames_df = df.groupby('Item ID')['Price'].count().sort_values(ascending = False).reset_index(name= "Purchase Count")
top5populargames_df= listofpopulargames_df.iloc[0:5,0:2]

In [34]:
#list of popular game Item ID
idnumofpopulargames = list(listofpopulargames_df.iloc[0:5,0])
#I previously tried to find the most popular game by Item Name, but that didn't work because some Item Names had multiple ids. 

In [35]:
#Filtered DF is by popular games - Table 2 of Item Names and ID#s

populargames_df = df.loc[df['Item ID'].isin(idnumofpopulargames)]

In [36]:
totalpurchval_mostpopular = populargames_df.groupby('Item ID')['Price'].sum().reset_index(name = "Total Purchase Value")

In [37]:
itemprice_mostpopular = populargames_df.groupby('Item ID')['Price'].mean().reset_index(name = "Purchase Price")

In [38]:
count_itemid_mostpopular = populargames_df.groupby('Item ID')['Item Name'].nunique().reset_index(name = "Item Name")

In [39]:
itemnames = populargames_df[['Item ID', 'Item Name']].drop_duplicates()

In [40]:
#merge most popluar games dfs
itemnamesandpurchcount_df= pd.merge(itemnames,top5populargames_df, how = 'outer', on = "Item ID")
itemprice_merge_df = pd.merge(itemnamesandpurchcount_df, itemprice_mostpopular, how = 'outer', on= "Item ID")
mostpopular_final_df = pd.merge(itemprice_merge_df,totalpurchval_mostpopular, how = 'outer', on = 'Item ID')
mostpopular_final_df.set_index("Item ID").reset_index()

,Item ID,Item Name,Purchase Count,Purchase Price,Total Purchase Value
0,108.00,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
1,92.00,Final Critic,8,4.88,39.04
2,82.00,Nirvana,9,4.90,44.10
3,178.00,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
4,145.00,Fiery Glass Crusader,9,4.58,41.22


In [41]:
#source[create a new column]: https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
#source[dloc by list]: https://stackoverflow.com/questions/45803676/python-pandas-loc-filter-for-list-of-values
#source[Selectcolumnsindf]:https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe
#source[Dropduplicaterows]:http://pandas.pydata.org/pandas-docs/version/0.17/generated/pandas.DataFrame.drop_duplicates.html

### Most Profitable Items

In [50]:
listofprofitgames_df = df.groupby('Item ID')['Price'].sum().sort_values(ascending = False).reset_index(name= "Total Purchase Value")
top5profitgames_df= listofprofitgames_df.iloc[0:5,0:2]

In [51]:
list_of_profit_games_df = list(top5profitgames_df.iloc[0:5,0])

In [52]:
profitgames_df = df.loc[df['Item ID'].isin(list_of_profit_games_df)]

In [53]:
profit_purchasecount_df = profitgames_df.groupby('Item ID')['Price'].count().reset_index(name ='Purchase Count')

In [54]:
itemnames_profit = profitgames_df[['Item ID', 'Item Name']].drop_duplicates()

In [55]:
itemprice_profit = profitgames_df.groupby('Item ID')['Price'].mean().reset_index(name ='Purchase Price')

In [56]:
#merge most popluar games dfs

profitable_itemnamesandpurchcount_df= pd.merge(itemnames_profit,profit_purchasecount_df, how = 'outer', on = "Item ID")
profitable_itemprice_merge_df = pd.merge(profitable_itemnamesandpurchcount_df, itemprice_profit, how = 'outer', on= "Item ID")
mostprofit_final_df = pd.merge(profitable_itemprice_merge_df,top5profitgames_df, how = 'outer', on = 'Item ID')
mostprofit_final_df.set_index("Item ID").reset_index()

,Item ID,Item Name,Purchase Count,Purchase Price,Total Purchase Value
0,92.00,Final Critic,8,4.88,39.04
1,82.00,Nirvana,9,4.90,44.10
2,178.00,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
3,145.00,Fiery Glass Crusader,9,4.58,41.22
4,103.00,Singed Scalpel,8,4.35,34.80


# <font color= green> Pandas Analysis: Conclusions </font>

***
#### Gender: 
- Men have more presence, yet woman pay more per order. 
- Other genders marginally purchase more than men. 
#### Age: 
- Ages 20 through 24 are the most active demographic. 
- Ages 35 through 40 spend more per purchase. 
- This may be an error: but the average purchase price and average purchase total are the same.
#### Profitability and Popularity: 
- 3 out of the 5 most propular games are the most profitable.

***

### <font color = blue>Extra Analysis </font>
#### Number of Item IDs per Item Name

In [58]:
#To confirm a unique Item ID per Item Name
#To highlight value greater than 1
df_itemsvsitemid= df.groupby('Item Name')['Item ID'].nunique().reset_index(name= "# of Item IDs")
#df_itemsvsitemid_id= df_itemsvsitemid.subset=['# of Item IDs']
df_itemsvsitemid_summarized = df_itemsvsitemid.loc[df_itemsvsitemid['# of Item IDs'] > 1]
df_itemsvsitemid_summarized

,Item Name,# of Item IDs
28,Crucifer,2
56,Final Critic,2
98,Persuasion,2
137,Stormcaller,2


In [315]:
#source[unqiue values per group]:https://stackoverflow.com/questions/41415017/count-unique-values-using-pandas-groupby

#### Table of Item IDs for Item Name: Crucifer

In [59]:
#to create a dataframe to review the item Ids for "Crucifer"
cruciferIDs_df=df.loc[df['Item Name'] == 'Crucifer']
cruciferIDs_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bracket
121,121.00,Ilosia37,23,Male,1.00,Crucifer,3.26,20-24
251,251.00,Iasur80,22,Male,23.00,Crucifer,1.99,20-24
454,454.00,Lassimla92,25,Male,1.00,Crucifer,3.26,25-29
697,697.00,Tyaelistidru84,19,Female,1.00,Crucifer,3.26,15-19


#### Table of Discounted Items

In [60]:
df_statsbyItem =round(df.groupby('Item Name')['Price'].describe().reset_index(),2).fillna(0)
df_statsbyItem_std = df_statsbyItem.loc[df_statsbyItem['std'] > .01]
df_statsbyItem_std

,Item Name,count,mean,std,min,25%,50%,75%,max
28,Crucifer,4.00,2.94,0.63,1.99,2.94,3.26,3.26,3.26
56,Final Critic,13.00,4.61,0.35,4.19,4.19,4.88,4.88,4.88
98,Persuasion,9.00,3.22,0.06,3.19,3.19,3.19,3.19,3.33
137,Stormcaller,3.00,2.59,0.66,2.21,2.21,2.21,2.78,3.36


#### Table of Items with an 'Item ID' of 23

In [61]:
#to review is Item ID # 23.0 is also for a another product
#maybe it was a misclassfication?

itemid23_df=df.loc[df['Item ID'] == 23]
itemid23_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bracket
251,251.00,Iasur80,22,Male,23.00,Crucifer,1.99,20-24


Since item 23 is only for Crucifer, it appears to be about a 40% discount value. <br>
Before I rename the Item Name for Item ID '23.0' I will check to see if other items had discounted items but with the same id # <br>
If so, then we will change the id # rather than the name

In [320]:
#def highlight_over0(df_statsbyItem, color='yellow'):
    #highlight a value greater than 0
   # print(df_statsbyItem)
   # attr= 'background-color: {}'.format(color)
  #  if (df_statsbyItem['std']>.01):
    #    highlight_over0 = df_statsbyItem == (df_statsbyItem['std']>.01)
    #    return [attr if (df_statsbyItem['std']>.01) else '' for df_statsbyItem in highlight_over0]
   # else:
      #  return [attr if df_statsbyItem(['std']>.01) else '' for df_statsbyItem in highlight_over0]

In [321]:
#res = df_statsbyItem.style.apply(highlight_over0, color= 'yellow', subset=['std'])
#res

#### Table of Discounted Items

In [62]:
#statistic summary on item name and prices
#to see products were purchased at discount
df_statsbyItem =round(df.groupby('Item Name')['Price'].describe().reset_index(),2).fillna(0)
df_statsbyItem_std = df_statsbyItem.loc[df_statsbyItem['std'] > .01]
df_statsbyItem_std

,Item Name,count,mean,std,min,25%,50%,75%,max
28,Crucifer,4.00,2.94,0.63,1.99,2.94,3.26,3.26,3.26
56,Final Critic,13.00,4.61,0.35,4.19,4.19,4.88,4.88,4.88
98,Persuasion,9.00,3.22,0.06,3.19,3.19,3.19,3.19,3.33
137,Stormcaller,3.00,2.59,0.66,2.21,2.21,2.21,2.78,3.36


We will rename Item ID 23 since this discount seems normal. The ID code should have remained the same. <br>
It could be an error in the sytem or a new disc code

In [63]:
df_rev1= df.loc[df['Item ID'] == 23].replace([23],1) 
df_rev1

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bracket
251,251.00,Iasur80,22,Male,1.00,Crucifer,1.99,20-24


In [ ]:
#source:https://stackoverflow.com/questions/52224142/pandas-conditionally-replace-values-based-on-other-columns-values

In [ ]:
#source[Name a df column]: https://stackoverflow.com/questions/29836477/pandas-create-new-column-with-count-from-groupby

In [34]:
#source[summarystatisticsbygroup]:https://datascience.stackexchange.com/questions/22266/summary-statistics-by-category-using-python